In [2]:
from getpass import getpass
token = getpass('GitHub token: ')

GitHub token: ··········


In [2]:
!git clone --branch feat/exp-generator https://$token@github.com/Bartekb3/fromscratch-transformer-classifier.git

fatal: destination path 'fromscratch-transformer-classifier' already exists and is not an empty directory.


In [5]:
%cd /content/fromscratch-transformer-classifier

/content/fromscratch-transformer-classifier


In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
if not (project_root / 'src').exists():
    print(f"⚠️ Ostrzeżenie: Nie znaleziono katalogu 'src' w {project_root}")
    
print(f"Dodano do ścieżki: {project_root}")
from src.textclf_transformer.tokenizer.wordpiece_tokenizer_wrapper import WordPieceTokenizerWrapper
from datasets import load_dataset
import sys
import numpy as np
tokenizer_dir = "src/textclf_transformer/tokenizer/BERT_original"

tokenizer = WordPieceTokenizerWrapper()
tokenizer.load(tokenizer_dir=tokenizer_dir)


Dodano do ścieżki: /Users/michaliwaniuk/fromscratch-transformer-classifier


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

ds = load_dataset(
    "wikimedia/wikipedia",
    "20231101.en",
    split="train",          # local Arrow dataset (not streaming)
    cache_dir="../drive/MyDrive/data/raw"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
import re

N = 600000
k = 3

sample = [None] * N

keywords = ["film", "movie", "cinema", "television", "series", "episode", "season", "actor", "actress", "director", "screenwriter", "producer", "soundtrack", "box", "office", "award", "academy", "oscar", "review", "reception", "critics", "plot", "culture", "algorithm", "theorem", "proof", "model", "dataset", "data", "method", "approach", "analysis", "experiment", "simulation", "complexity", "optimization", "neural", "network", "machine", "learning", "statistics", "statistical", "physics", "quantum", "entropy", "differential", "equation", "mathematics", "computer", "engineering", "artificial", "intelligence", "algorithmic", "election", "politics", "political", "party", "ideology", "government", "policy", "law", "immigration", "climate", "abortion", "gun", "control", "foreign", "propaganda", "media", "bias", "populism", "nationalism", "controversy", "criticism", "debate", "movement", "public", "opinion"]

kw = set(w.lower() for w in keywords)
token_re = re.compile(r"[a-z]+")  # English letters; adjust if you need hyphens/apostrophes/etc.

n = 0
for i, el in enumerate(ds):
    text = el["text"]
    low = text.lower()

    found = 0
    seen = set()  # ensure "distinct keywords" matched

    for tok in token_re.findall(low):
        if tok in kw and tok not in seen:
            seen.add(tok)
            found += 1
            if found >= k:
                sample[n] = text
                n += 1
                break

    if n == N:
        break

    # printing every iteration is *very* slow; print occasionally
    if (i & 8191) == 0:  # every 8192 docs
        print(i, n)



In [13]:
len(sample)

600000

In [ ]:
def shorten_texts(sample, max_words=600):
    shortened = []
    i=0
    for text in sample:
        i+=1
        if i%10000==0:
          print(i)
        words = text.split()
        shortened.append(" ".join(words[:max_words]))
    return shortened
shorten_sample = shorten_texts(sample)

In [16]:
import pickle

with open("../drive/MyDrive/data/raw/wikipedia.pkl", "wb") as f:
    pickle.dump(shorten_sample, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("../drive/MyDrive/data/raw/wikipedia.txt", "w", encoding="utf-8") as f:
    i=0
    for line in shorten_sample:
        i+=1
        if i%10000==0:
          print(i)
        f.write(line + "\n")

In [1]:
import pickle
with open("../../data/raw/wikipedia.pkl", "rb") as f:
    shorten_sample = pickle.load(f)

In [7]:
with open("../drive/MyDrive/data/raw/wikipedia.txt", "r", encoding="utf-8") as f:
    shorten_sample = f.read().splitlines()

In [ ]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import TensorDataset, ConcatDataset
tokenizer = WordPieceTokenizerWrapper()
tokenizer.load(tokenizer_dir='')

p1 = shorten_sample[:200000]
p2 = shorten_sample[200000:400000]
p3 = shorten_sample[400000:]

encoded1 = tokenizer.encode(p1, max_length=512)
encoded2 = tokenizer.encode(p2, max_length=512)
encoded3 = tokenizer.encode(p3, max_length=512)

concat = ConcatDataset([encoded1, encoded2, encoded3])

input_ids_list = []
attention_mask_list = []

for ds in concat.datasets:
    ids, mask = ds.tensors
    input_ids_list.append(ids)
    attention_mask_list.append(mask)

input_ids = torch.cat(input_ids_list, dim=0)
attention_mask = torch.cat(attention_mask_list, dim=0)
encoded = TensorDataset(input_ids, attention_mask)


In [26]:
encoded[:450000][0][:, 127] = 102 #[SEP]
encoded[:450000][0][:, 128:] = 0 #[PAD]
encoded[:450000][1][:, 128:] = True #[MASK]


In [35]:
from pathlib import Path
import torch
save_dir = Path("../../data/tokenized")
save_dir.mkdir(parents=True, exist_ok=True)

torch.save(encoded, save_dir / "wikipedia_train.pt")